In [66]:
import pandas as pd 
import numpy as np
import os


processed_dir = r"C:\Users\Students\Desktop\flax_projects\SDI_Tanzania_Healthcare_Delays\data\processed"
os.makedirs(processed_dir, exist_ok=True)

In [67]:
#Load providers dataset
provider_path = r"C:\Users\Students\Desktop\flax_projects\SDI_Tanzania_Healthcare_Delays\data\raw\provider_data.csv"
provider_data = pd.read_csv(provider_path)
provider_data.head()


,facility_id,provider_id,country,year,unique_id,num_staff,num_med,num_nonmed,provider_post1,provider_cadre1,...,provider_cadre2,provider_educ2,provider_mededuc2,provider_male2,provider_present2,absent_reason2,provider_activity,salary_delay,has_roster,has_absentee
0,1,1,TANZANIA,2014,1_1,1,1,1,1,2,...,2.0,2.0,3.0,1.0,1.0,NaN,1.0,0.0,1,1.0
1,1,2,TANZANIA,2014,1_2,1,1,1,5,4,...,4.0,2.0,2.0,0.0,1.0,NaN,1.0,0.0,1,1.0
2,1,3,TANZANIA,2014,1_3,1,1,1,3,4,...,4.0,2.0,2.0,0.0,1.0,NaN,1.0,0.0,1,1.0
3,10,5,TANZANIA,2014,10_5,2,2,1,5,4,...,4.0,1.0,2.0,0.0,1.0,NaN,1.0,0.0,1,1.0
4,10,8,TANZANIA,2014,10_8,2,2,1,5,4,...,4.0,1.0,2.0,0.0,1.0,NaN,1.0,0.0,1,1.0


In [68]:
provider_data.shape
provider_data.info()



<class 'pandas.DataFrame'>
RangeIndex: 5267 entries, 0 to 5266
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   facility_id        5267 non-null   int64  
 1   provider_id        5267 non-null   int64  
 2   country            5267 non-null   str    
 3   year               5267 non-null   int64  
 4   unique_id          5267 non-null   str    
 5   num_staff          5267 non-null   int64  
 6   num_med            5267 non-null   int64  
 7   num_nonmed         5267 non-null   int64  
 8   provider_post1     5267 non-null   int64  
 9   provider_cadre1    5267 non-null   int64  
 10  provider_educ1     5266 non-null   float64
 11  provider_mededuc1  5261 non-null   float64
 12  provider_male1     5267 non-null   int64  
 13  is_outpatient      5267 non-null   int64  
 14  provider_present1  5267 non-null   int64  
 15  absent_reason1     2292 non-null   float64
 16  provider_post2     2571 non-null   

In [69]:
provider_data['facility_id'].nunique(), provider_data.shape[0]


(401, 5267)

In [70]:
core_provider_cols = [
    'facility_id',
    'provider_present1',   # presence indicator
    'provider_male1',      # gender
    'provider_post1',      # post
    'provider_cadre1',     # cadre / profession
    'provider_educ1',      # education level
    'provider_mededuc1',   # medical education
    'num_staff',           # total staff in facility
    'num_med',             # number of medical staff
    'num_nonmed',          # non-medical staff
    'has_roster',          # management indicator
    'salary_delay'         # salary issues
]


In [71]:
provider_core = provider_data[core_provider_cols].copy()
provider_core.head()

,facility_id,provider_present1,provider_male1,provider_post1,provider_cadre1,provider_educ1,provider_mededuc1,num_staff,num_med,num_nonmed,has_roster,salary_delay
0,1,1,1,1,2,2.0,3.0,1,1,1,1,0.0
1,1,1,0,5,4,2.0,2.0,1,1,1,1,0.0
2,1,1,0,3,4,2.0,2.0,1,1,1,1,0.0
3,10,1,0,5,4,1.0,2.0,2,2,1,1,0.0
4,10,1,0,5,4,1.0,2.0,2,2,1,1,0.0


In [72]:
agg_dict = {
    'provider_present1': 'mean',
    'provider_male1': 'mean',
    'provider_post1': lambda x: x.mode()[0] if not x.mode().empty else None,
    'provider_cadre1': lambda x: x.mode()[0] if not x.mode().empty else None,
    'provider_educ1': lambda x: x.mode()[0] if not x.mode().empty else None,
    'provider_mededuc1': 'mean',
    'num_staff': 'max',
    'num_med': 'max',
    'num_nonmed': 'max',
    'has_roster': lambda x: x.mode()[0] if not x.mode().empty else None,
    'salary_delay': 'mean'
}

provider_facility = provider_core.groupby('facility_id').agg(agg_dict).reset_index()


In [73]:
provider_facility.head()

,facility_id,provider_present1,provider_male1,provider_post1,provider_cadre1,provider_educ1,provider_mededuc1,num_staff,num_med,num_nonmed,has_roster,salary_delay
0,1,1.000,0.333333,1,4,2.0,2.333333,1,1,1,1,0.0
1,2,1.000,1.000000,1,2,2.0,2.500000,1,1,1,1,0.0
2,3,0.875,0.250000,3,4,2.0,2.500000,2,2,1,1,0.0
3,4,1.000,0.000000,1,4,2.0,2.000000,1,1,0,1,0.0
4,5,0.500,0.500000,1,3,1.0,2.000000,1,1,1,1,0.0


In [74]:

provider_facility.shape  # should be (401, 12)


(401, 12)

In [75]:
# Save provider_facility
provider_facility_path = os.path.join(processed_dir, "provider_facility.csv")
provider_facility.to_csv(provider_facility_path, index=False)

In [76]:
os.listdir(processed_dir)

['facility_core.csv', 'facility_full.csv', 'provider_facility.csv']

In [77]:
provider_facility.isna().sum()


facility_id          0
provider_present1    0
provider_male1       0
provider_post1       0
provider_cadre1      0
provider_educ1       0
provider_mededuc1    0
num_staff            0
num_med              0
num_nonmed           0
has_roster           0
salary_delay         0
dtype: int64

In [78]:
provider_facility.to_csv(r'C:\Users\Students\Desktop\flax_projects\SDI_Tanzania_Healthcare_Delays\data\processed\provider_facility.csv', index=False)
